> # **Extract Single Table from PDF File URL Using Python**

**OBJECTIVE**
- Given a PDF file comprising of text, figures and tables, our goal is to extract a specific table of interest from a specified page. 
- We have a PDF rul of interest, the UN World Population Prospects 2019 Highlights. 
- Specifically, our task is to extract Table 1. population of the world, SDG regions and selected groups of countries, 2019, 2030, 2050 and 2100, according to the medium- variant projection.

In [198]:
# First verify whether the needed libraries and their dependencies are part of Python builtin packages in google colab
!pip list -v | grep tabula-py
!pip list -v | grep pandas
!pip list -v | grep numpy
!pip list -v | grep urllib

tabula-py                     2.5.1                        /usr/local/lib/python3.7/dist-packages pip
pandas                        1.3.5                        /usr/local/lib/python3.7/dist-packages pip
pandas-datareader             0.9.0                        /usr/local/lib/python3.7/dist-packages pip
pandas-gbq                    0.13.3                       /usr/local/lib/python3.7/dist-packages pip
pandas-profiling              1.4.1                        /usr/local/lib/python3.7/dist-packages pip
sklearn-pandas                1.8.0                        /usr/local/lib/python3.7/dist-packages pip
numpy                         1.21.6                       /usr/local/lib/python3.7/dist-packages pip
urllib3                       1.24.3                       /usr/local/lib/python3.7/dist-packages pip


In [199]:
# If no builtin libraries and their dependencies, install or upgrade Required Libraries
# !pip install -U tabula-py    #to scrape text from PDF files
# !pip install -U pandas       #to construct and manipulate our panel data
# !pip install -U numpy
# !pip install -U urllib


In [200]:
# Import libraries into Python session
import tabula as tb
import pandas as pd
import numpy as np
import urllib
from urllib import request
import sys
import dataframe_image as dfi

In [201]:
# Absolute URL of PDF File 
file='https://population.un.org/wpp/Publications/Files/WPP2019_Highlights.pdf'

In [205]:
# Check website access policies

# importing robot parser class
import urllib.robotparser as rb
  
bot = rb.RobotFileParser()
  
# checks where the website's robot.txt file reside
x = bot.set_url('https://population.un.org/robot.txt')
print('robot.txt file presence:', x)
  
# reads the files
y = bot.read()
print('requests can read robots.txt files:', y)
  
# we can crawl the main site
z = bot.can_fetch('*', 'https://population.un.org/')
print('crawl maiin site:', z)
  
# but can not crawl the disallowed url
w = bot.can_fetch('*', 'https://population.un.org/wp-admin/')
print('cannot crawl the disallowed url:', w)

robot.txt file presence: None
requests can read robots.txt files: None
crawl maiin site: True
cannot crawl the disallowed url: True


In [206]:
# Check for valid URL to continue to terminate program

def check_validity(file):
    try:
        urllib.request.urlopen(file)
        print("Valid File URL")
    except IOError:
        print ("Invalid File URL")
        sys.exit()
check_validity(file)

Valid File URL


In [207]:
#Use tabula to read all tables in a specified index page of web pdf file, which is 35 for our case.
file='https://population.un.org/wpp/Publications/Files/WPP2019_Highlights.pdf'
data = tb.read_pdf(file, pages = '12')

#result is stored in list that determinen the class of functions and attributes we can apply. check type.
type(data)


list

In [208]:
# turn list into a dataframe
# you can use numpy helper to do this, because if you convert a list directly into a dataframe in a way
# df = pd.DataFrame(data) # it can never be done! you will get an error: ValueError: Must pass 2-d input.

# you have to first create a concancated array from the list, data

df = pd.DataFrame(np.concatenate(data))
df


,0,1,2,3,4
0,Region,2019,2030,2050,2100
1,World,7 713,8 548,9 735,10 875
2,Sub-Saharan Africa,1 066,1 400,2 118,3 775
3,Northern Africa and Western Asia,517,609,754,924
4,Central and Southern Asia,1 991,2 227,2 496,2 334
5,Eastern and South-Eastern Asia,2 335,2 427,2 411,1 967
6,Latin America and the Caribbean,648,706,762,680
7,Australia/New Zealand,30,33,38,49
8,Oceania*,12,15,19,26
9,Europe and Northern America,1 114,1 132,1 136,1 120


In [209]:
#count missing values in feature
pd.isnull(df)


,0,1,2,3,4
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
5,False,False,False,False,False
6,False,False,False,False,False
7,False,False,False,False,False
8,False,False,False,False,False
9,False,False,False,False,False


In [210]:
#Drop rows with any NaN values
df.dropna(inplace=True)

#Verify all rows containing at least one NA have been dropped successfully
df


,0,1,2,3,4
0,Region,2019,2030,2050,2100
1,World,7 713,8 548,9 735,10 875
2,Sub-Saharan Africa,1 066,1 400,2 118,3 775
3,Northern Africa and Western Asia,517,609,754,924
4,Central and Southern Asia,1 991,2 227,2 496,2 334
5,Eastern and South-Eastern Asia,2 335,2 427,2 411,1 967
6,Latin America and the Caribbean,648,706,762,680
7,Australia/New Zealand,30,33,38,49
8,Oceania*,12,15,19,26
9,Europe and Northern America,1 114,1 132,1 136,1 120


In [211]:
# The results above may be sufficient but unfortunately it is not what we really expected, as we know that this table makes everything in it is stored in rows without having a name header for each column.
# In this case, according to the table in the file, we will use Region and Years as header references for each column.

# convert first row to column header
df.columns = df.iloc[0] 

# We now have colum header and first row being the same. So drop first row
df = df.drop(0)

#We see that the row index starts from 1 instead of the standard 0. So, lets reset current row index to the default index.
df = df.reset_index(drop=True)

#print dataframe
print(df)

#create excel workbook in current working directory
df.to_excel("output.xlsx")

#specify a sheet name to save extracted table
df.to_excel("output.xlsx", sheet_name="Table 1")

0                             Region   2019   2030   2050    2100
0                              World  7 713  8 548  9 735  10 875
1                 Sub-Saharan Africa  1 066  1 400  2 118   3 775
2   Northern Africa and Western Asia    517    609    754     924
3          Central and Southern Asia  1 991  2 227  2 496   2 334
4     Eastern and South-Eastern Asia  2 335  2 427  2 411   1 967
5    Latin America and the Caribbean    648    706    762     680
6              Australia/New Zealand     30     33     38      49
7                           Oceania*     12     15     19      26
8        Europe and Northern America  1 114  1 132  1 136   1 120
9          Least developed countries  1 033  1 314  1 877   3 047
10  Land-locked Developing Countries    521    659    926   1 406
11    Small Island Developing States     71     78     87      88
